In [10]:
import nltk
from nltk import FreqDist, word_tokenize
import string
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

In [4]:
stream_content_df = pd.read_csv("data/stream_content.csv", header=0, encoding="ISO-8859-1")
stream_content_df.head()

,StreamID,Content
0,163,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...
1,419,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...
2,507,"wBYKUgUyGWc\r\nA team of world-class drivers, ..."
3,199,Castrol EDGE is Castrol?s flagship power bran...
4,201,"Charles ?Cheers? Wakefield, Castrol?s founder,..."


## Pre-process the content to remove stop words, punctutations and lemmatization

In [9]:
def preprocess(tokens):
    
    # TODO: remove random sequences that contain with more than one caps and small or combination of letters and numbers
    
    tokens_nop = [t for t in tokens if t not in string.punctuation]
    tokens_nop = [t.lower() for t in tokens_nop]
    wnl = nltk.WordNetLemmatizer()
    stop = stopwords.words('english')
    tokens_nostop = [t for t in tokens_nop if t not in stop]
    tokens_lem = [wnl.lemmatize(t) for t in tokens_nostop]
    tokens_clean = [t for t in tokens_lem if len(t) >= 3] 
    return tokens_clean

stream_content_df['Content_processed'] = stream_content_df['Content'].map(word_tokenize)
stream_content_df['Content_processed'] = stream_content_df.Content_processed.apply(preprocess)
stream_content_df.head()

,StreamID,Content,Content_processed
0,163,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...,"[txmak2kzay4, nmeujebo1ac, eeutxfhp3go, castro..."
1,419,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...,"[txmak2kzay4, nmeujebo1ac, eeutxfhp3go, castro..."
2,507,"wBYKUgUyGWc\r\nA team of world-class drivers, ...","[wbykuguygwc, team, world-class, driver, power..."
3,199,Castrol EDGE is Castrol?s flagship power bran...,"[castrol, edge, castrol, flagship, power, bran..."
4,201,"Charles ?Cheers? Wakefield, Castrol?s founder,...","[charles, cheer, wakefield, castrol, founder, ..."


### Generate the TFIDF vectors for the streams

In [11]:
stream_content_df['Content_processed'] = stream_content_df['Content_processed'].apply(lambda x: " ".join(x))
stream_content_df.head()

,StreamID,Content,Content_processed
0,163,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...,txmak2kzay4 nmeujebo1ac eeutxfhp3go castrol la...
1,419,TXmAk2KZAy4\r\nNMeUjebo1Ac\r\nEEuTxFhp3go\r\nC...,txmak2kzay4 nmeujebo1ac eeutxfhp3go castrol la...
2,507,"wBYKUgUyGWc\r\nA team of world-class drivers, ...",wbykuguygwc team world-class driver powered ca...
3,199,Castrol EDGE is Castrol?s flagship power bran...,castrol edge castrol flagship power brand pcos...
4,201,"Charles ?Cheers? Wakefield, Castrol?s founder,...",charles cheer wakefield castrol founder entrep...


In [14]:
all_streams_cleaned_text = stream_content_df['Content_processed']
all_streams_tfidf_vectorizer = TfidfVectorizer(min_df = 2)
all_streams_tfidf = all_streams_tfidf_vectorizer.fit_transform(all_streams_cleaned_text)
all_streams_tfidf

<97x928 sparse matrix of type '<class 'numpy.float64'>'
	with 6122 stored elements in Compressed Sparse Row format>

In [16]:
token_values = {all_streams_tfidf_vectorizer.vocabulary_[token]: token for token in all_streams_tfidf_vectorizer.vocabulary_}


In [20]:
similarities = cosine_similarity(all_streams_tfidf)

In [30]:
def linear(x, total_steps):
    step_value = 1/total_steps
    return 1 - (x * step_value)

In [31]:
def constant(x, total_steps):
    return 1

In [34]:
def get_similar_streams_based_on_history(viewed_streams, weight_pattern = linear, max_similar_streams = 10, max_viewed_streams_to_consider = 5):
    """
        viewed_streams: list of stream IDs that have been viewed by the current user. The streams at a lower ID has been viewed more recently. So stream with ID 0 is the last viewed stream.
        weight_pattern: The weight pattern to weight the contributions due to the stream history
    max_similar_streams: The maximum number of similar streams to return
    max_viewed_streams_to_consider: The maximum number of viewed streams to consider for the recommendation 
    
    """
    
    if viewed_streams:
        num_viewed_streams = len(viewed_streams)
        
        # set the max viewed streams to consider 
        max_viewed_streams_to_consider = min(max_viewed_streams_to_consider, num_viewed_streams)
        
        # create an array of 0's
        similarity_sum = np.zeros(similarities.shape[0])
    
        for x, viewed_stream in enumerate(viewed_streams):
            stream_index = stream_content_df[stream_content_df["StreamID"]==viewed_stream].index[0]
            weight_factor = weight_pattern(x, max_viewed_streams_to_consider)
            print(weight_factor)
            similarity_sum = similarity_sum + weight_factor * np.array(similarities[stream_index])
            
        print(similarity_sum)
            

In [36]:
get_similar_streams_based_on_history([163, 507, 201])

1
1
1
[1.26842638 1.26842638 1.26653507 0.54206015 1.08246848 1.08060241
 0.38248413 0.20116951 0.14658722 0.29725233 0.10648097 0.08576749
 0.02097258 0.01210118 0.03825771 0.15016732 0.34640278 0.17474736
 0.1207582  0.0363342  0.0306509  0.13868422 0.11879601 0.26807067
 0.14191606 0.05777261 0.15773115 0.30005363 0.26230676 0.32719142
 0.20784978 0.1399473  0.05196305 0.54929018 0.54519433 0.5631664
 0.52810759 0.08714075 0.21471683 0.18800053 0.20505879 0.19026916
 0.17775237 0.12416441 0.13972612 0.14364081 0.58097796 0.12020355
 0.17359848 0.05299836 0.09932326 0.13450363 0.11263966 0.04197468
 0.08477123 0.11959233 0.52442963 0.07549113 0.18922715 0.11582825
 0.91592479 0.13137204 0.06632045 0.02042509 0.49038357 0.49502248
 0.17080126 0.19891587 0.1374694  0.19450259 0.16737659 0.49844015
 0.4838611  0.0629477  0.07123764 0.02045516 0.06119652 0.06107664
 0.19197842 0.15410387 0.47337929 0.25946396 0.19040369 0.11001998
 0.0493245  0.54123731 0.13443178 0.15992612 0.07392323 0